In [1]:
import numpy as np


In [2]:
xTemp = np.zeros((1000, 905760))
with open("C:\\Users\\yangz\\Desktop\\FireProjectBatchDownloadNew\\DetectionDatasetFinalFlattened\\TrainingA_1-1000.npy", 'rb') as f1:
        xTemp = np.load(f1)
        

In [3]:
yTemp = np.zeros((1000))
for i in range (1000):
    if (i%2==0):
        yTemp[i] = 0
    else:
        yTemp[i] = 1


In [4]:
x = np.zeros((1000, 905760))
y = np.zeros((1000))

permutations = np.random.permutation(1000)
for i in range (1000):
    x[i] = xTemp[permutations[i]]
    y[i] = yTemp[permutations[i]]
    

In [5]:
validation = np.zeros((212, 905720))
with open("C:\\Users\\yangz\\Desktop\\FireProjectBatchDownloadNew\\DetectionDatasetFinalFlattened\\Validation_1908-2120.npy", 'rb') as f1:
        validation = np.load(f1)

In [6]:
yValidation = np.zeros((212))
for i in range (212):
    if (i%2==0):
        yValidation[i] = 0
    else:
        yValidation[i] = 1
        

In [7]:
from sklearn.ensemble import RandomForestClassifier
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
from sklearn.metrics import roc_auc_score
import sys

space = {
            'estimators': hp.uniform('estimators', 1, 20),
    
            'max_depth': hp.uniform('max_depth', 2, 15),
    
            'min_samples_split': hp.uniform('min_samples_split', 20, 500),
    
            'min_samples_leaf': hp.uniform('min_samples_leaf', 100, 400),
            
        }

In [8]:
from tqdm.keras import TqdmCallback

def f_nn(params):
    
    clf = RandomForestClassifier(n_estimators=int(params['estimators']), max_depth=int(params['max_depth']), min_samples_split=int(params['min_samples_split']), min_samples_leaf=int(params['min_samples_leaf']))
    clf.fit(x, y)
    
    yPred = clf.predict(validation)
    yPred[yPred>=0.5] = 1
    yPred[yPred<0.5] = 0
    yPred = yPred.astype(int)
    yPred = yPred.flatten()
    
    correct = 0
    for i in range (212):
        if (yPred[i]==yValidation[i]):
            correct+=1
    acc = correct/212
    
    print('Accuracy:', acc)
    sys.stdout.flush() 
    return {'loss': -acc, 'status': STATUS_OK}
    
trials = Trials()
best = fmin(f_nn, space, algo=tpe.suggest, max_evals=20, trials=trials)
print('best: ', best)

Accuracy:                                                                           
0.6886792452830188                                                                  
Accuracy:                                                                           
0.7405660377358491                                                                  
Accuracy:                                                                           
0.7405660377358491                                                                  
Accuracy:                                                                           
0.6933962264150944                                                                  
Accuracy:                                                                           
0.7264150943396226                                                                  
Accuracy:                                                                           
0.7641509433962265                                               

In [ ]:
#Acc1: 0.7025862068965517 / 0.7122641509433962